In [1]:
import sympy as sy
sy.init_printing()

In [95]:
b = sy.symbols(['x','y','z', 'w', 'p', 'k'])
ins = sy.symbols(['X','Y','Z', 'W', 'P', 'K'])
pt = sy.symbols(['px', 'py', 'pz'])
plane = sy.symbols(['a', 'b', 'c', 'd'])

# Make the Rotation matrix

In [47]:
R_x = sy.Matrix([[1,0,0],
               [0, sy.cos(ins[3]), -1*sy.sin(ins[3])], 
               [0, sy.sin(ins[3]), sy.cos(ins[3])]])

R_y = sy.Matrix([[sy.cos(ins[4]),0,sy.sin(ins[4])],
               [0, 1, 0], 
               [-1*sy.sin(ins[4]), 0, sy.cos(ins[4])]])

R_z = sy.Matrix([[sy.cos(ins[5]),-1*sy.sin(ins[5]),0],
               [sy.sin(ins[5]), sy.cos(ins[5]), 0], 
               [0, 0, 1]])

In [55]:
R_x

⎡1    0        0   ⎤
⎢                  ⎥
⎢0  cos(W)  -sin(W)⎥
⎢                  ⎥
⎣0  sin(W)  cos(W) ⎦

In [56]:
R_y

⎡cos(P)   0  sin(P)⎤
⎢                  ⎥
⎢   0     1    0   ⎥
⎢                  ⎥
⎣-sin(P)  0  cos(P)⎦

In [57]:
R_z

⎡cos(K)  -sin(K)  0⎤
⎢                  ⎥
⎢sin(K)  cos(K)   0⎥
⎢                  ⎥
⎣  0        0     1⎦

In [58]:
R = R_z*R_y*R_x

In [59]:
R

⎡cos(K)⋅cos(P)  -sin(K)⋅cos(W) + sin(P)⋅sin(W)⋅cos(K)  sin(K)⋅sin(W) + sin(P)⋅
⎢                                                                             
⎢sin(K)⋅cos(P)  sin(K)⋅sin(P)⋅sin(W) + cos(K)⋅cos(W)   sin(K)⋅sin(P)⋅cos(W) - 
⎢                                                                             
⎣   -sin(P)                 sin(W)⋅cos(P)                         cos(P)⋅cos(W

cos(K)⋅cos(W)⎤
             ⎥
sin(W)⋅cos(K)⎥
             ⎥
)            ⎦

In [73]:
R_x = sy.Matrix([[1,0,0],
               [0, sy.cos(b[3]), -1*sy.sin(b[3])], 
               [0, sy.sin(b[3]), sy.cos(b[3])]])

R_y = sy.Matrix([[sy.cos(b[4]),0,sy.sin(b[4])],
               [0, 1, 0], 
               [-1*sy.sin(b[4]), 0, sy.cos(b[4])]])

R_z = sy.Matrix([[sy.cos(b[5]),-1*sy.sin(b[5]),0],
               [sy.sin(b[5]), sy.cos(b[5]), 0], 
               [0, 0, 1]])
R_b =  R_z*R_y*R_x

# Make the vectors

In [97]:
r_eop = sy.Matrix([ins[0],ins[1],ins[2]])
r_b = sy.Matrix([b[0],b[1],b[2]])
r_pt = sy.Matrix([pt[0],pt[1],pt[2]])
r_plane = sy.Matrix([plane[0],plane[1],plane[2]])

In [98]:
fx_1 = r_eop + R*r_b + R*R_b*r_pt
fx = fx_1.dot(r_plane) + plane[3]

# Do the Derivitives

In [112]:
# d/d b
d_fx_bx = sy.Derivative(fx,b[0] ).doit()
d_fx_by = sy.Derivative(fx,b[1] ).doit()
d_fx_bz = sy.Derivative(fx,b[2] ).doit()

d_fx_bw = sy.Derivative(fx,b[3] ).doit()
d_fx_bp = sy.Derivative(fx,b[4] ).doit()
d_fx_bk = sy.Derivative(fx,b[5] ).doit()

# d/d plane
d_fx_a = sy.Derivative(fx,plane[0] ).doit()
d_fx_b = sy.Derivative(fx,plane[1] ).doit()
d_fx_c = sy.Derivative(fx,plane[2] ).doit()
d_fx_d = sy.Derivative(fx,plane[3] ).doit()


In [119]:
d_fx_bx